In [ ]:
!pip install catboost
!pip install nonconformist

In [4]:
import numpy as np
import pandas as pd
import logging
from sklearn.preprocessing import MinMaxScaler, StandardScaler, KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from joblib import dump

# Initialize logging



In [ ]:
import xgboost as xgb
from sklearn.calibration import CalibratedClassifierCV
from nonconformist.cp import IcpClassifier
from nonconformist.nc import MarginErrFunc, ClassifierNc

# 1. Modeli Eğitin ve Kalibre Edin

# XGBoost sınıflandırıcısını oluşturun
clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# Eğitim verisiyle modeli eğitin
clf.fit(X_train, y_train)

# Modeli kalibre edin
calibrated_clf = CalibratedClassifierCV(clf, method='sigmoid', cv='prefit')
calibrated_clf.fit(X_val, y_val)

# 2. Conformal Prediction Uygulayın

# Uygunluk puanlarını hesaplamak için nonconformist kütüphanesini kullanın
nc = ClassifierNc(calibrated_clf, MarginErrFunc())
icp = IcpClassifier(nc)

# Eğitim ve doğrulama verisini birleştirin
icp.fit(X_train, y_train)
icp.calibrate(X_val, y_val)

# X_test üzerinde güven aralıkları oluşturun
predictions = icp.predict(X_test, significance=0.05)


In [ ]:
# X_test üzerinde %95 güven seviyesinde tahminler al
predictions = icp.predict(X_test, significance=0.05)

# Tahminlerin doğru olup olmadığını kontrol edin
correct = (predictions[:, 0] == y_test) & (predictions[:, 1] == True)

# Doğru tahminlerin oranını hesaplayın
accuracy = correct.sum() / len(y_test)

# Tahminlerin genişliğini değerlendirin (eğer uygundur)
widths = predictions[:, 2] - predictions[:, 0]

average_width = widths.mean()

print(f"Accuracy at 95% confidence: {accuracy * 100:.2f}%")
print(f"Average prediction interval width: {average_width:.2f}")


In [ ]:
# X_test üzerinde %95 güven seviyesinde tahminler al
predictions = icp.predict(X_test, significance=0.05)

# Tahminlerin yapıldığı veri noktalarının oranını hesaplayın
prediction_coverage = (predictions[:, 1] == True).sum() / len(y_test)

print(f"Prediction coverage at 95% confidence: {prediction_coverage * 100:.2f}%")
